In [1]:
import pandas as pd
import numpy as np
import random, os, re, sys, scipy, matplotlib, string, nltk, nltk.stem
import matplotlib.pyplot as plt
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc  ###计算roc和auc
import networkx as nx

In [2]:
f = open('DATA/API_PT.txt', 'r', encoding='utf-8')
API_pt_dic = {}
for line in f.readlines():
    words = re.split('\t', line[:-1])
    API_pt_dic[words[0]] = words[1]
f.close()
print(API_pt_dic)

{'2checkout': '2checkout', 'admob': 'admob', 'allegro': 'allegro', 'amazon-ec2': 'amazon-ec2', 'amazon-queue-service': 'amazon-sqs', 'amazon-s3': 'amazon-s3', 'amazon-ses': 'amazon-ses', 'amazon-sns': 'amazon-sns', 'authorizenet': 'authorize.net', 'aws-cloud9': 'aws-cloud9', 'bigcommerce': 'bigcommerce', 'bing-maps': 'bing-maps', 'breadcrumbs': 'breadcrumbs', 'brightcove': 'brightcove', 'buffer': 'buffer', 'cartodb': 'cartodb', 'coinbase': 'coinbase-api', 'coupon': 'coupon', 'dailymotion': 'dailymotion-api', 'deezer': 'deezer', 'digitalocean': 'digital-ocean', 'dropbox': 'dropbox-api', 'eventbrite': 'eventbrite', 'facebook-ads': 'facebook-ads-api', 'facebook': 'facebook', 'flickr': 'flickr', 'foursquare': 'foursquare', 'geocoder': 'google-geocoder', 'geonames': 'geonames', 'gmail': 'gmail-api', 'google-adsense': 'adsense-api', 'google-adwords': 'google-adwords', 'google-app-engine': 'google-app-engine', 'google-checkout': 'google-checkout', 'google-cloud-dns': 'google-cloud-dns', 'goog

In [3]:
api_path = 'DATA/clean_API_data/'
api_list = os.listdir(api_path)
for i in range(0, len(api_list)):
    print(i, api_list[i])

0 clean_google-checkout.csv
1 clean_google-adwords.csv
2 clean_wurfl.csv
3 clean_deezer.csv
4 clean_itunes-and-itunes-connect.csv
5 clean_musicbrainz.csv
6 clean_facebook.csv
7 clean_zipcode.csv
8 clean_google-directions.csv
9 clean_flickr.csv
10 clean_dailymotion.csv
11 clean_mailgun.csv
12 clean_tumblr.csv
13 clean_mapbox.csv
14 clean_lastfm.csv
15 clean_yahoo-mail.csv
16 clean_amazon-s3.csv
17 clean_aws-cloud9.csv
18 clean_twitter.csv
19 clean_reddit.csv
20 clean_brightcove.csv
21 clean_pinterest.csv
22 clean_twilio.csv
23 clean_woocommerce.csv
24 clean_geocoder.csv
25 clean_cartodb.csv
26 clean_google-cloud-sql.csv
27 clean_coupon.csv
28 clean_google-adsense.csv
29 clean_soundcloud.csv
30 clean_gravatar.csv
31 clean_google-earth.csv
32 clean_google-fusion-tables.csv
33 clean_google-cloud-dns.csv
34 clean_google-plus.csv
35 clean_openlayers.csv
36 clean_plunker.csv
37 clean_google-maps-android.csv
38 clean_ilist.csv
39 clean_allegro.csv
40 clean_openstreetmap.csv
41 clean_paypal.csv

# 用于比较不同API的SO正例中Tag的分布差异性，进一步表明通过Tag来筛选API正样本数据的合理性

## 依次处理API

In [8]:
tag_index_dic = {}
api_tags_dic = {}
ps_dic = {}

In [9]:
cnt = 0
for index in range(0, len(api_list)):
    api_name = api_list[index][6:-4]
    print(api_name)
    df = pd.DataFrame(pd.read_csv(api_path+api_list[index], header=0))
    pt = '<'+API_pt_dic[api_name]+'>'
    tag_cnt_dic = {}
    for i in range(0, df.shape[0]):
        if i%1000 == 0:
            print('i:', i)
        if pt in df.iloc[i].Tags:
            if api_name in ps_dic:
                ps_dic[api_name] += 1
            else:
                ps_dic[api_name] = 1
            tags = re.findall('<.*?>', df.iloc[i].Tags, re.S)
            for j in range(0, len(tags)):
                tags[j] = tags[j][1:-1]
                if tags[j] not in tag_index_dic:
                    tag_index_dic[tags[j]] = cnt
                    cnt += 1
                if tags[j] not in tag_cnt_dic:
                    tag_cnt_dic[tags[j]] = 1
                else:
                    tag_cnt_dic[tags[j]] += 1
    api_tags_dic[api_name] = tag_cnt_dic

google-checkout
i: 0
i: 1000
google-adwords
i: 0
i: 1000
i: 2000
wurfl
i: 0
deezer
i: 0
itunes-and-itunes-connect
i: 0
i: 1000
i: 2000
i: 3000
i: 4000
i: 5000
i: 6000
i: 7000
i: 8000
musicbrainz
i: 0
facebook
i: 0
i: 1000
i: 2000
i: 3000
i: 4000
i: 5000
i: 6000
i: 7000
i: 8000
i: 9000
i: 10000
i: 11000
i: 12000
i: 13000
i: 14000
i: 15000
i: 16000
i: 17000
i: 18000
i: 19000
i: 20000
i: 21000
i: 22000
i: 23000
i: 24000
i: 25000
i: 26000
i: 27000
i: 28000
i: 29000
i: 30000
i: 31000
i: 32000
i: 33000
i: 34000
i: 35000
i: 36000
i: 37000
i: 38000
i: 39000
i: 40000
i: 41000
i: 42000
i: 43000
i: 44000
i: 45000
i: 46000
i: 47000
i: 48000
i: 49000
i: 50000
i: 51000
i: 52000
i: 53000
i: 54000
i: 55000
i: 56000
i: 57000
i: 58000
i: 59000
i: 60000
i: 61000
i: 62000
i: 63000
i: 64000
i: 65000
i: 66000
i: 67000
i: 68000
i: 69000
i: 70000
i: 71000
i: 72000
i: 73000
i: 74000
i: 75000
i: 76000
i: 77000
i: 78000
i: 79000
i: 80000
i: 81000
i: 82000
i: 83000
i: 84000
i: 85000
i: 86000
i: 87000
i: 88000
i: 

i: 30000
i: 31000
i: 32000
i: 33000
i: 34000
i: 35000
i: 36000
i: 37000
i: 38000
i: 39000
i: 40000
i: 41000
i: 42000
i: 43000
i: 44000
i: 45000
i: 46000
i: 47000
i: 48000
i: 49000
i: 50000
i: 51000
i: 52000
i: 53000
i: 54000
i: 55000
i: 56000
i: 57000
i: 58000
i: 59000
i: 60000
i: 61000
i: 62000
i: 63000
i: 64000
i: 65000
i: 66000
i: 67000
i: 68000
i: 69000
i: 70000
i: 71000
i: 72000
i: 73000
i: 74000
i: 75000
i: 76000
i: 77000
i: 78000
i: 79000
i: 80000
i: 81000
i: 82000
i: 83000
i: 84000
i: 85000
i: 86000
i: 87000
i: 88000
i: 89000
i: 90000
i: 91000
i: 92000
i: 93000
i: 94000
i: 95000
i: 96000
i: 97000
i: 98000
i: 99000
i: 100000
i: 101000
i: 102000
i: 103000
i: 104000
i: 105000
i: 106000
i: 107000
i: 108000
i: 109000
i: 110000
i: 111000
i: 112000
i: 113000
i: 114000
heatmap
i: 0
i: 1000
i: 2000
i: 3000
i: 4000
xero
i: 0
google-cloud-print
i: 0
multimap
i: 0
i: 1000
google-static-maps
i: 0
i: 1000
google-drive
i: 0
i: 1000
i: 2000
i: 3000
i: 4000
i: 5000
i: 6000
i: 7000
i: 8000
i: 90

In [10]:
for key in api_tags_dic:
    print(key, ps_dic[key])
    for tag in api_tags_dic[key]:
        api_tags_dic[key][tag] /= ps_dic[key]

google-checkout 374
google-adwords 1653
wurfl 114
deezer 306
itunes-and-itunes-connect 4852
musicbrainz 99
facebook 83229
zipcode 353
google-directions 291
flickr 1167
dailymotion 203
mailgun 1176
tumblr 2020
mapbox 3471
lastfm 285
yahoo-mail 139
amazon-s3 27732
aws-cloud9 91
twitter 22347
reddit 807
brightcove 253
pinterest 828
twilio 1059
woocommerce 19295
geocoder 913
cartodb 152
google-cloud-sql 2045
coupon 305
google-adsense 97
soundcloud 2056
gravatar 219
google-earth 846
google-fusion-tables 1369
google-cloud-dns 127
google-plus 4201
openlayers 3294
plunker 255
google-maps-android 3309
ilist 288
allegro 356
openstreetmap 2980
paypal 18252
digitalocean 2108
facebook-ads 728
rackspace-cloud-servers 332
urban-airship 631
2checkout 135
amazon-sns 1540
imgur 424
breadcrumbs 956
bigcommerce 1276
marketo 240
youtube 9212
heroku 30439
amazon-queue-service 316
mailjet 123
mandrill 921
instagram-graph 78
opensocial 167
recurly 149
openstack 2329
pubnub 946
bing-maps 2417
eventbrite 271
fo

In [11]:
api_vector_dic = {}
for apiname in api_tags_dic:
    print(apiname)
    vt = []
    for tagname in tag_index_dic:
        if tagname in api_tags_dic[apiname]:
            vt.append(api_tags_dic[apiname][tagname])
        else:
            vt.append(0)
    api_vector_dic[apiname] = vt

google-checkout
google-adwords
wurfl
deezer
itunes-and-itunes-connect
musicbrainz
facebook
zipcode
google-directions
flickr
dailymotion
mailgun
tumblr
mapbox
lastfm
yahoo-mail
amazon-s3
aws-cloud9
twitter
reddit
brightcove
pinterest
twilio
woocommerce
geocoder
cartodb
google-cloud-sql
coupon
google-adsense
soundcloud
gravatar
google-earth
google-fusion-tables
google-cloud-dns
google-plus
openlayers
plunker
google-maps-android
ilist
allegro
openstreetmap
paypal
digitalocean
facebook-ads
rackspace-cloud-servers
urban-airship
2checkout
amazon-sns
imgur
breadcrumbs
bigcommerce
marketo
youtube
heroku
amazon-queue-service
mailjet
mandrill
instagram-graph
opensocial
recurly
openstack
pubnub
bing-maps
eventbrite
foursquare
plivo
google-mirror
google-maps-places
telegram-bot
sendgrid
amazon-ses
coinbase
onelogin
google-app-engine
linkedin
mixer
vimeo
payments-gateway
yammer
sms
authorizenet
buffer
heatmap
xero
google-cloud-print
multimap
google-static-maps
google-drive
rhapsody
vk
google-street

In [12]:
apiname_list = list(api_vector_dic.keys())

In [13]:
sims = []
for i in range(0, len(apiname_list)):
    for j in range(i+1, len(apiname_list)):
        x = api_vector_dic[apiname_list[i]]
        y = api_vector_dic[apiname_list[j]]
        d3 = np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
        sims.append(d3)

In [14]:
ave = 0
for ele in sims:
    ave += ele
print('余弦相似度平均值：', ave/len(sims))
print('余弦相似度最大值：', max(sims))
print('余弦相似度最小值：', min(sims))

余弦相似度平均值： 0.03935385215703903
余弦相似度最大值： 0.588894762509578
余弦相似度最小值： 0.0


以上实验结果表明，不同API之间的Tag分布确实存在差异，具有明显的区分度。

# 比较同一API正负样本之间Tag的分布差异

为了消除正样本中positive tag的影响，需要就算去除pos-tag之后的分布差异。  
理想情况下，无标记样本中含有一些正样本，因此相似度不会很低；又由于无标记样本中含有负样本，因此相似度应该比正样本之间相互的相似度要低。  
因此需要比较pos-pos的相似度和pos-unL的相似度差异。

In [4]:
def save_tag_to_dic(tags, tags_list_dic, dic, pt):
    for i in range(0, len(tags)):
        if tags[i] != pt:  ## ingore the positive tag
            tags[i] = tags[i][1:-1]
            if tags[i] not in dic:
                tags_list_dic[tags[i]] = 1
                dic[tags[i]] = 1
            else:
                dic[tags[i]] += 1
    return tags_list_dic, dic

In [5]:
def cal_sim(tags_list_dic, dic1, n1, dic2, n2):
    sps = []
    sns = []
    for key in tags_list_dic:
        if key in dic1:
            sps.append(dic1[key]/n1)
        else:
            sps.append(0)
        if key in dic2:
            sns.append(dic2[key]/n2)
        else:
            sns.append(0)
    return np.dot(sps, sns) / (np.linalg.norm(sps) * np.linalg.norm(sns))

In [4]:
import api_tags_similarity

pp_sims = []
pu_sims = []
for index in range(0, len(api_list)):
    api_name = api_list[index][6:-4]
    tags_list_dic = {}
    df = pd.DataFrame(pd.read_csv(api_path+api_list[index], header=0))
    print(index, api_name, df.shape[0])
    pt = '<'+API_pt_dic[api_name]+'>'
    print(pt)
    ps = 0
    ns = 0
    ps1 = 0
    ps2 = 0
    pts_dic = {}
    nts_dic = {}
    pts_dic1 = {}
    pts_dic2 = {}
    
    for i in range(0, df.shape[0]):
        tags = df.iloc[i].Tags
        tags = re.findall('<.*?>', tags, re.S)
        if pt in tags:
            ps += 1
            tags_list_dic, pts_dic = api_tags_similarity.save_tag_to_dic(tags, tags_list_dic, pts_dic, pt)
            if ps%2 == 0:
                ps1 += 1
                tags_list_dic, pts_dic1 = api_tags_similarity.save_tag_to_dic(tags, tags_list_dic, pts_dic1, pt)
            else:
                ps2 += 1
                tags_list_dic, pts_dic2 = api_tags_similarity.save_tag_to_dic(tags, tags_list_dic, pts_dic2, pt)
        else:
            ns += 1
            tags_list_dic, nts_dic = api_tags_similarity.save_tag_to_dic(tags, tags_list_dic, nts_dic, pt)
    print(ps1, ps2)
    pu_sims.append(api_tags_similarity.cal_sim(tags_list_dic, pts_dic, ps, nts_dic, ns))
    pp_sims.append(api_tags_similarity.cal_sim(tags_list_dic, pts_dic1, ps1, pts_dic2, ps2))

0 google-checkout 1596
<google-checkout>
187 187
1 google-adwords 2135
<google-adwords>
826 827
2 wurfl 190
<wurfl>
57 57
3 deezer 363
<deezer>
153 153
4 itunes-and-itunes-connect 8226
<itunesconnect>
2426 2426
5 musicbrainz 177
<musicbrainz>
49 50
6 facebook 148956
<facebook>
41614 41615
7 zipcode 3706
<zipcode>
176 177
8 google-directions 13692
<google-direction>
145 146
9 flickr 2957
<flickr>
583 584
10 dailymotion 456
<dailymotion-api>
101 102
11 mailgun 1592
<mailgun>
588 588
12 tumblr 3319
<tumblr>
1010 1010
13 mapbox 4602
<mapbox>
1735 1736
14 lastfm 612
<last.fm>
142 143
15 yahoo-mail 3626
<yahoo-mail>
69 70
16 amazon-s3 31507
<amazon-s3>
13866 13866
17 aws-cloud9 192
<aws-cloud9>
45 46
18 twitter 157891
<twitter>
11173 11174
19 reddit 3428
<reddit>
403 404
20 brightcove 331
<brightcove>
126 127
21 pinterest 2053
<pinterest>
414 414
22 twilio 6679
<twilio-api>
529 530
23 woocommerce 21381
<woocommerce>
9647 9648
24 geocoder 3957
<google-geocoder>
456 457
25 cartodb 221
<cartodb

In [5]:
ave = 0
for ele in pu_sims:
    ave += ele
print('余弦相似度平均值：', ave/len(pu_sims))
print('余弦相似度最大值：', max(pu_sims))
print('余弦相似度最小值：', min(pu_sims))

余弦相似度平均值： 0.8277232361526983
余弦相似度最大值： 0.9876339888309276
余弦相似度最小值： 0.040491754232484634


In [6]:
ave = 0
for ele in pp_sims:
    ave += ele
print('余弦相似度平均值：', ave/len(pp_sims))
print('余弦相似度最大值：', max(pp_sims))
print('余弦相似度最小值：', min(pp_sims))

余弦相似度平均值： 0.9521124164107548
余弦相似度最大值： 0.9998427431405879
余弦相似度最小值： 0.7433788270884971


In [75]:
for i in range(len(pu_sims)):
    print(pu_sims[i], pp_sims[i])

0.7326330275058284 0.9264559962287924
0.9276639360811434 0.9759204565491575
0.8889814643493604 0.872846252221172
0.6326827709813673 0.9620175507674567
0.9743686499721779 0.9992081430448262
0.7766930104747466 0.7972623559582612
0.9059476118384924 0.9997035139723383
0.7891299601711002 0.9039063407603509
0.8617868221659593 0.9812142950084153
0.8984765742686137 0.9646726177841568
0.8906410657496012 0.8376117599510814
0.9633307261399604 0.9742345785377687
0.9471662686490627 0.9880385087049963
0.8449480280654024 0.9937736470172875
0.9076587947554982 0.9459698628691127
0.8899589887817525 0.935707924868464
0.9130043123740936 0.9995227557476442
0.902903608759105 0.8966806358471773
0.2715968545288838 0.9979962998329459
0.7932431312828527 0.9766538645737421
0.8637184939552446 0.9270762143553146
0.8643969038364739 0.9667242335863754
0.9821863153067774 0.9977813367552492
0.976919200219089 0.9998427431405879
0.8443640214274021 0.9824203000247045
0.8516838059736793 0.8411124938762624
0.91149858169588

验证结果表明，Tag用来作分类是可行的